In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('transaction_dataset.csv')

In [3]:
df.head(10)

,tr_id,p_id,c_id,tr_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,25-02-2017 00:00,False,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0
1,2,3,3120,21-05-2017 00:00,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0
2,3,37,402,16-10-2017 00:00,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0
3,4,88,3135,31-08-2017 00:00,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0
4,5,78,787,01-10-2017 00:00,True,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0
5,6,25,2339,08-03-2017 00:00,True,Approved,Giant Bicycles,Road,medium,medium,1538.99,829.65,39031.0
6,7,22,1542,21-04-2017 00:00,True,Approved,WeareA2B,Standard,medium,medium,60.34,45.26,34165.0
7,8,15,2459,15-07-2017 00:00,False,Approved,WeareA2B,Standard,medium,medium,1292.84,13.44,39915.0
8,9,67,1305,10-08-2017 00:00,False,Approved,Solex,Standard,medium,large,1071.23,380.74,33455.0
9,10,12,3262,30-08-2017 00:00,True,Approved,WeareA2B,Standard,medium,medium,1231.15,161.60,38216.0


In [4]:
df.drop(['product_class', 'product_size'], axis=1, inplace=True)

In [5]:
df.rename(columns={'tr_id': 'transaction_id',
                   'p_id': 'product_id',
                   'c_id': 'customer_id',
                   'tr_date': 'transaction_date'}, inplace=True)

In [6]:
df.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,list_price,standard_cost,product_first_sold_date
0,1,2,2950,25-02-2017 00:00,False,Approved,Solex,Standard,71.49,53.62,41245.0
1,2,3,3120,21-05-2017 00:00,True,Approved,Trek Bicycles,Standard,2091.47,388.92,41701.0
2,3,37,402,16-10-2017 00:00,False,Approved,OHM Cycles,Standard,1793.43,248.82,36361.0
3,4,88,3135,31-08-2017 00:00,False,Approved,Norco Bicycles,Standard,1198.46,381.10,36145.0
4,5,78,787,01-10-2017 00:00,True,Approved,Giant Bicycles,Standard,1765.30,709.48,42226.0


In [7]:
df.to_csv('cleaned_transaction_dataset.csv', index=False)

### Filtering Approved Orders

In [8]:
approved_df = df[df['order_status'] == 'Approved']

In [9]:
approved_df.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,list_price,standard_cost,product_first_sold_date
0,1,2,2950,25-02-2017 00:00,False,Approved,Solex,Standard,71.49,53.62,41245.0
1,2,3,3120,21-05-2017 00:00,True,Approved,Trek Bicycles,Standard,2091.47,388.92,41701.0
2,3,37,402,16-10-2017 00:00,False,Approved,OHM Cycles,Standard,1793.43,248.82,36361.0
3,4,88,3135,31-08-2017 00:00,False,Approved,Norco Bicycles,Standard,1198.46,381.10,36145.0
4,5,78,787,01-10-2017 00:00,True,Approved,Giant Bicycles,Standard,1765.30,709.48,42226.0


### Data Processing and Month Indexing

In [10]:
from datetime import datetime


filtered_df = approved_df[['customer_id', 'transaction_date']].copy()

# Convert the 'transaction_date' column to datetime using df[df.columns[i]] = newvals
filtered_df['transaction_date'] = pd.to_datetime(filtered_df['transaction_date'])

# Create a new 'YM' column using explicit column name
filtered_df['YM'] = filtered_df['transaction_date'].apply(lambda x: int(x.strftime('%Y%m')))

# Determine the start month using .min() on the 'YM' column
start_month = filtered_df['YM'].min()

# Calculate a new column 'transaction_month_index' using explicit column name
filtered_df['transaction_month_index'] = filtered_df['YM'] - start_month

In [11]:
filtered_df.head()

,customer_id,transaction_date,YM,transaction_month_index
0,2950,2017-02-25,201702,1
1,3120,2017-05-21,201705,4
2,402,2017-10-16,201710,9
3,3135,2017-08-31,201708,7
4,787,2017-01-10,201701,0


### Cohort Identification and Month Index Reset

In [12]:
cohort_month = filtered_df.groupby('customer_id')['transaction_month_index'].min().reset_index()

cohort_month.rename(columns={'transaction_month_index': 'cohort_month'}, inplace=True)


In [13]:
# Merging 'filtered_df' and 'cohort_month' based on the 'customer_id' column
data = filtered_df.join(cohort_month.set_index('customer_id'), on='customer_id')


In [14]:
# Creating a new column 'cohort_index' by subtracting 'cohort_month' from 'transaction_month_index'
data['cohort_index'] = data['transaction_month_index'] - data['cohort_month']

In [15]:
data.head()

,customer_id,transaction_date,YM,transaction_month_index,cohort_month,cohort_index
0,2950,2017-02-25,201702,1,1,0
1,3120,2017-05-21,201705,4,0,4
2,402,2017-10-16,201710,9,0,9
3,3135,2017-08-31,201708,7,1,6
4,787,2017-01-10,201701,0,0,0


In [16]:
df1 = data[['customer_id', 'cohort_month', 'cohort_index']]

df2 = df1.drop_duplicates(subset=['customer_id', 'cohort_month', 'cohort_index'])

# Generating 'final_df' by grouping the unique rows in 'df2' and counting unique customers within each cohort group
final_df = df2.groupby(['cohort_month', 'cohort_index']).agg({'customer_id': 'nunique'}).reset_index()

final_df.rename(columns={'customer_id': 'customer_count'}, inplace=True)

In [17]:
final_df.head()

,cohort_month,cohort_index,customer_count
0,0,0,1343
1,0,1,489
2,0,2,484
3,0,3,478
4,0,4,481


In [18]:
# Creating a pivot table 'cohort_data' from 'final_df'
cohort_data = final_df.pivot_table(index='cohort_month', columns='cohort_index', values='customer_count')

In [19]:
cohort_data.head()

cohort_index,0,1,2,3,4,5,6,7,8,9,10,11
cohort_month,,,,,,,,,,,,
0,1343.0,489.0,484.0,478.0,481.0,480.0,526.0,507.0,478.0,511.0,504.0,498.0
1,802.0,322.0,305.0,328.0,299.0,296.0,322.0,304.0,330.0,308.0,280.0,NaN
2,502.0,190.0,182.0,190.0,188.0,217.0,196.0,188.0,191.0,204.0,NaN,NaN
3,308.0,102.0,110.0,128.0,120.0,97.0,131.0,130.0,130.0,NaN,NaN,NaN
4,215.0,84.0,89.0,86.0,80.0,90.0,66.0,81.0,NaN,NaN,NaN,NaN


In [20]:
# Calculating 'cohort_percentage' by dividing each value in 'cohort_data' by the values in the first column
cohort_percentage = cohort_data.divide(cohort_data.iloc[:, 0], axis=0)

cohort_percentage = (cohort_percentage * 100).round(3)

cohort_percentage

cohort_index,0,1,2,3,4,5,6,7,8,9,10,11
cohort_month,,,,,,,,,,,,
0,100.0,36.411,36.039,35.592,35.815,35.741,39.166,37.751,35.592,38.049,37.528,37.081
1,100.0,40.150,38.030,40.898,37.282,36.908,40.150,37.905,41.147,38.404,34.913,NaN
2,100.0,37.849,36.255,37.849,37.450,43.227,39.044,37.450,38.048,40.637,NaN,NaN
3,100.0,33.117,35.714,41.558,38.961,31.494,42.532,42.208,42.208,NaN,NaN,NaN
4,100.0,39.070,41.395,40.000,37.209,41.860,30.698,37.674,NaN,NaN,NaN,NaN
5,100.0,31.356,40.678,42.373,33.898,40.678,38.983,NaN,NaN,NaN,NaN,NaN
6,100.0,52.500,26.250,37.500,43.750,32.500,NaN,NaN,NaN,NaN,NaN,NaN
7,100.0,35.185,37.037,40.741,25.926,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,100.0,35.294,26.471,47.059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
